### The Search for Mjolnir 
Using data on thunderstorms and meteorite strikes, this notebook finds the most likely locations of Mjolnir, the mystical hammer of Thor.
Note: Thunder is the 2nd digit of 'FRSHTT', so thunder_df=weather_data[(weather_data['FRSHTT']//10)%2==1] would capture all instances of thunder

In [3]:
import pandas as pd
import math
import numpy as np

pd.options.mode.copy_on_write = True

# mile range of bounding box for finding stations to check thunder
mile_range=10
#constant value for determining thunderstorms above the average
thunder_over_avg=10
thunder_over_avg_ratio=1.5

#start and end dates to analyze as a list. combinations based on output from Thunder_data_per_station are in comments below
#start dates [1929,1970,1980,1990,2000,2010,2020]
file_start_dates=[2020]
#end dates[1969,1979,1989,1999,2009,2019,2025]
file_end_dates=[2025]

def geo_box(lat,long):
    """returns a list of 4 values for latitude and longitude that creates a bounding box
    a certain amount of miles away from the supplied longitute and latitude points
    as determined by the global variable 'mile_range'
    output: [min_lat, max_lat, min_long, max_long]"""
    global  mile_range
    def haversine(angle):
        return math.sin(angle/2)**2
    def archaversine(angle):
        return 2*math.asin(math.sqrt(angle))
    earth_radius=3963.19 #radius of earth in miles
    rad_dist=mile_range/earth_radius
    lat_rad=lat*math.pi/180
    long_rad=long*math.pi/180
    coord_list_rad=[0.0,0.0,0.0,0.0]
    #latitute calculations and place in list
    coord_list_rad[0]=lat_rad-rad_dist
    coord_list_rad[1]=lat_rad+rad_dist
    #longitude calculations and place in list 
    long_rad_calc=archaversine(haversine(rad_dist)/math.cos(lat_rad)**2)
    coord_list_rad[2]=long_rad-long_rad_calc
    coord_list_rad[3]=long_rad+long_rad_calc
    #convert from radians back to degrees
    coord_list_deg=[coord_list_rad[0]*180/math.pi,coord_list_rad[1]*180/math.pi,
                    coord_list_rad[2]*180/math.pi,coord_list_rad[3]*180/math.pi]
    return coord_list_deg

#read and clean meteorite data
meteorite_data = pd.read_csv('Meteorite_Landings.csv')
meteorite_data_clean=meteorite_data.dropna(subset=['year','GeoLocation','reclat','reclong'],ignore_index=True)
meteorite_data_clean=meteorite_data_clean[meteorite_data_clean['GeoLocation']!="(0.0, 0.0)"].reset_index(drop=True)

for i in range(0,len(file_start_dates)):
        #start and end years to analyze
    year_start=file_start_dates[i]
    year_end=file_end_dates[i]
    print(f'\nCalculating meteorite data year range {year_start}-{year_end} ------ {i+1}/{len(file_start_dates)} year sets')
    #create dictionary of dataframes with the key being the year for each dataframe
    weather_df_dict={}
    for year in range(year_start,year_end+1):
        print(f"\rLoading file: {year}_data.csv            ", end = "", flush = True)
        weather_df_dict[year]=pd.read_csv(f'Weather_data/{year}_data.csv')
        weather_df_dict[year].dropna(ignore_index=True,inplace=True)
        # if stations_lightning_info.empty:
        #     stations_lightning_info['NAME']=weather_df_dict[year]['NAME'].unique()
    print('Complete')

    #load station data
    station_data = pd.read_csv(f'Thunder_data_{year_start}_{year_end}.csv')

    #create dataframe that only contains meteorites that fell in the range of years specified
    meteorite_data_in_range=meteorite_data_clean[(meteorite_data_clean['year']>=year_start) & (meteorite_data_clean['year']<=year_end)].reset_index(drop=True)
    
    #define new dataframe to contain just meteorites that have associated stations
    meteorite_data_with_stations=pd.DataFrame(columns=meteorite_data_in_range.columns)
    meteorite_data_with_stations['Stations_in_Range']=None
    meteorite_data_with_stations['Thunder_Instances']=None
    meteorite_data_with_stations['Area_Thunder_Average']=None
    meteorite_data_with_stations['Landing_thunder_over_avg']=None

    #if no meteorites are found in the range of years, print in terminal and save an empty .csv file
    if meteorite_data_in_range.empty:
        print(f'No Meteorites Found between {year_start} and {year_end}')
        meteorite_data_with_stations.to_csv(f'Meteorite_Analysis_{year_start}to{year_end}_{mile_range}miles_{thunder_over_avg}a{thunder_over_avg_ratio}thunder.csv',index=False)
        print(f'Meteorite_Analysis_{year_start}to{year_end}_{mile_range}miles_{thunder_over_avg}a{thunder_over_avg_ratio}thunder.csv save complete.')
        continue

    #calculate the min and max latitude and longitude to create a square box around each meteorite in which to find stations
    #uses the geo_box function defined above to calculate values which can be checked for each station
    meteorite_data_in_range[['min_lat','max_lat','min_long','max_long']]=meteorite_data_in_range.apply(lambda x:
        geo_box(x['reclat'],x['reclong']),axis=1,result_type='expand')

    #loop through meteorites to find stations that are in range to hear thunder
    for index, row in meteorite_data_in_range.iterrows():
        print(f'\rAnalyzing meteorite: {index}/{len(meteorite_data_in_range)-1}        ',end="",flush=True)
        fall_year=row['year']
        #create dataframe of all stations close enough for thunder to be heard from the meteor landing point
        #this is a box around the landing point with each side "mile_range" from the landing point
        stations_in_range=weather_df_dict[fall_year][(weather_df_dict[fall_year]['LATITUDE']>=row['min_lat'])&
                    (weather_df_dict[fall_year]['LATITUDE']<=row['max_lat'])&
                    (weather_df_dict[fall_year]['LONGITUDE']>=row['min_long'])&
                    (weather_df_dict[fall_year]['LONGITUDE']<=row['max_long'])]
        #stations_in_range.head()
        row['Stations_in_Range']=stations_in_range[['NAME','LATITUDE','LONGITUDE']].drop_duplicates(subset=['NAME']).to_dict(orient='records')
        #fill new dataframe with only meteorites that have associated stations
        if stations_in_range.empty==False:
            #find all local stations with thunder with unique dates for storms
            stations_with_thunder=stations_in_range[(stations_in_range['FRSHTT']//10)%2==1]
            row['Thunder_Instances']=len(stations_with_thunder['DATE'].unique())
            #create average Thunder instances per year from an average of all stations in range
            station_list=stations_in_range['NAME'].unique().tolist()
            average_thunder=[]
            for station in station_list:
                average_thunder.append(station_data.loc[station_data['NAME']==station,'WEIGHTED_AVERAGE'].tolist()[0])
            row['Area_Thunder_Average']=sum(average_thunder)/len(average_thunder)
            if row['Area_Thunder_Average']==0:
                row['Landing_thunder_over_avg']=0
            else:
                row['Landing_thunder_over_avg']=row['Thunder_Instances']/row['Area_Thunder_Average']
            #fill new dataframe with only meteorites that have associated stations
            #and if that year's thunderstorms are a certain amount over the average
            if row['Thunder_Instances']>=row['Area_Thunder_Average']+thunder_over_avg and row['Landing_thunder_over_avg']>=thunder_over_avg_ratio:
                meteorite_data_with_stations.loc[len(meteorite_data_with_stations)]=row
    print('Complete')
    meteorite_data_with_stations.to_csv(f'Meteorite_Analysis_{year_start}to{year_end}_{mile_range}miles_{thunder_over_avg}a{thunder_over_avg_ratio}thunder.csv',index=False)
    print(f'Meteorite_Analysis_{year_start}to{year_end}_{mile_range}miles_{thunder_over_avg}a{thunder_over_avg_ratio}thunder.csv save complete.')
print('Complete')

#debugging output
#meteorite_data_in_range.info()
#meteorite_data_with_stations.info()
#print(meteorite_data_with_stations.describe())
#print(meteorite_data_with_stations.head())




Calculating meteorite data year range 2020-2025 ------ 1/1 year sets
Loading file: 2025_data.csv            Complete
No Meteorites Found between 2020 and 2025
Meteorite_Analysis_2020to2025_10miles_10a1.5thunder.csv save complete.
Complete
